# Operating Room Scheduling

This was an assignment from my Optimization course. It tasked us to write and code an optimization model to minimize the total under-allocation of each medical department to operating rooms in a hospital.

In [15]:
from gurobipy import *

## Part (a)

Let `x_ijk` be a binary decision variable that is 1 if department `i` uses room `j` on day `k` or else 0, and `h_jk` be the number of hours for room `j` on day `k`.

For department `i`, let `a_i` be the new total number of hours allocated each week, `t_i` be the target number of hours allocated each week, and `p_i` be the penalty or the total underallocation in number of hours.

The optimization model can be written as follows:

\begin{align}
\underset{p_i}{\text{min}} \;\; & \sum_{i = 1}^6 p_i \;/\; t_i \\
\text{s.t.} \;\; \text{New total number of hours:} \\
& \sum_{j=1}^5 \sum_{k=1}^5 x_{ijk} * h_{jk} = a_{i}, \quad i = 1,2,\ldots,6, \\
\text{Room-day exclusive constraint:} \\
& \sum_{i=1}^6 x_{ijk} \le 1, \quad j = 1,2,\ldots,5, \;\; k = 1,2,\ldots,5, \\
\text{Underallocation constraint:} \\
& p_{i} \ge 0, \quad i = 1,2,\ldots,6, \\
& p_{i} \ge t_{i} - a_{i}, \quad i = 1,2,\ldots,6, \\
\text{Binary decision variables:} \\
& x_{ijk} \in \{0,1\} \quad i = 1,\ldots,6, \;\; j = 1,2,\ldots,5, \;\; k = 1,2,\ldots,5.
\end{align}

### Construct variables

In [16]:
m = 6   # department
n = 5   # room
p = 5   # day

# row is room index, column is day index
hours_matrix = [[9,9,9,9,9],
                [9,9,9,9,8],
                [9,9,9,9,8],
                [9,9,9,9,8],
                [7.5,7.5,7.5,7.5,6.5]]

# target hours for departments
target_hours = [103.3, 9.0, 54.0, 15.8, 11.3, 20.3]

In [17]:
# construct a 'blank' model
mod = Model()

# define decision variables
x = mod.addVars(m, n, p, vtype = GRB.BINARY)
new_hours = mod.addVars(m)
penalty = mod.addVars(m)

### Construct constraints

In [18]:
# New hours for each department is the cross product of the hours matrix and the decision variable matrix
for i in range(m):
    mod.addConstr(new_hours[i] == sum(hours_matrix[j][k] * x[i,j,k] for j in range(n) for k in range(p))) 

In [19]:
# Penalty for under allocation
for i in range(m):
    mod.addConstr(penalty[i] >= 0)
    mod.addConstr(penalty[i] >= target_hours[i] - new_hours[i])

In [20]:
# Exclusive: at most 1 department can be assigned to room j on day k
for j in range(n):
    for k in range(p):
        mod.addConstr(sum(x[i,j,k] for i in range(m)) <= 1)

### Create the objective function, and set it to be minimized

In [21]:
mod.setObjective(sum(penalty[i] / target_hours[i] for i in range(m)), GRB.MINIMIZE)

In [22]:
mod.update()

mod.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 43 rows, 162 columns and 324 nonzeros
Model fingerprint: 0x800a5cc2
Variable types: 12 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Presolve removed 12 rows and 6 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 0 continuous, 156 integer (150 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 1.936108e-03, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [23]:
opt_val = mod.objval
print("Minimal penalty:", opt_val)

Minimal penalty: 0.05130687318489838


#### The schedule can be seen below:

In [24]:
for i in range(m):
    print(f"Department {i}")
    print(f"   R1  R2  R3  R4  R5")
    for k in range(p):
        
        if k == 0:
            output = "M"
        elif k == 1:
            output = "T"
        elif k == 2:
            output = "W" 
        elif k == 3:
            output = "T"
        elif k == 4:
            output = "F"
            
        for j in range(n):
            output += f"   {int(x[i,j,k].x)}"
        print(output)

Department 0
   R1  R2  R3  R4  R5
M   1   1   0   1   0
T   0   1   1   0   0
W   0   1   0   0   0
T   1   1   1   0   0
F   1   0   0   1   0
Department 1
   R1  R2  R3  R4  R5
M   0   0   0   0   0
T   0   0   0   0   0
W   0   0   1   0   0
T   0   0   0   0   0
F   0   0   0   0   0
Department 2
   R1  R2  R3  R4  R5
M   0   0   1   0   0
T   1   0   0   1   0
W   1   0   0   1   0
T   0   0   0   1   0
F   0   0   0   0   0
Department 3
   R1  R2  R3  R4  R5
M   0   0   0   0   0
T   0   0   0   0   0
W   0   0   0   0   0
T   0   0   0   0   0
F   0   1   1   0   0
Department 4
   R1  R2  R3  R4  R5
M   0   0   0   0   1
T   0   0   0   0   0
W   0   0   0   0   0
T   0   0   0   0   0
F   0   0   0   0   1
Department 5
   R1  R2  R3  R4  R5
M   0   0   0   0   0
T   0   0   0   0   1
W   0   0   0   0   1
T   0   0   0   0   1
F   0   0   0   0   0


## Part (b)

The hospital CEO inquired a schedule so that no department is split between two or more floors on the same day. 

Let `v_ikl` be a binary decision variable that is 1 if department `i` uses room(s) on floor `l` on day `k` or else 0.

The optimization model will have the following additional constraints:

\begin{align}
\text{Relationship between rooms and floors:} \\
& 2 * v_{ik1} >= x_{i1k} + x_{i2k}, \quad i = 1,2,\ldots,6, \;\; k = 1,2,\ldots,5,\\
& 2 * v_{ik2} >= x_{i3k} + x_{i4k}, \quad i = 1,2,\ldots,6, \;\; k = 1,2,\ldots,5,\\
& v_{ik3} >= x_{i5k}, \quad i = 1,2,\ldots,6, \;\; k = 1,2,\ldots,5,\\
\text{Department-day exclusive constraint:} \\
& \sum_{l=1}^3 v_{ikl} \le 1, \quad i = 1,2,\ldots,6, \;\; k = 1,2,\ldots,5, \\
\text{Binary decision variables:} \\
& v_{ikl} \in \{0,1\} \quad i = 1,\ldots,6, \;\; k = 1,2,\ldots,5, \;\; l = 1,2,3.
\end{align}

### Construct variables

In [25]:
q = 3 # floor
v = mod.addVars(m, p, q, vtype = GRB.BINARY)

### Construct constraints

In [26]:
# Relationship between floor and rooms

for i in range(m):
    for k in range(p):
        mod.addConstr(2 * v[i,k,0] >= x[i,0,k] + x[i,1,k])
        mod.addConstr(2 * v[i,k,1] >= x[i,2,k] + x[i,3,k])
        mod.addConstr(v[i,k,2] >= x[i,4,k])
        
        
# Exclusive: at most 1 floor assigned to the same department on the same day
for i in range(m):
    for k in range(p):
        mod.addConstr(sum(v[i,k,l] for l in range(q)) <= 1)

### Create the objective function, and set it to be minimized

In [27]:
mod.setObjective(sum(penalty[i] / target_hours[i] for i in range(m)), GRB.MINIMIZE)

In [28]:
mod.update()

mod.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 163 rows, 252 columns and 654 nonzeros
Model fingerprint: 0xf954a02b
Variable types: 12 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve did not produce a new incumbent solution

Found heuristic solution: objective 6.0000000
Presolve removed 72 rows and 66 columns
Presolve time: 0.00s
Presolved: 91 rows, 186 columns, 516 nonzeros
Variable types: 0 continuous, 186 integer (180 binary)
Found heuristic solution: objective 5.9141123

Root relaxation: objective 1.384318e-01, 106 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Une

In [29]:
opt_val = mod.objval
print("Minimal penalty:", opt_val)

Minimal penalty: 0.13843175217812198


#### The schedule can be seen below:

In [30]:
for i in range(m):
    print(f"Department {i}")
    print(f"   R1  R2  R3  R4  R5")
    for k in range(p):
        
        if k == 0:
            output = "M"
        elif k == 1:
            output = "T"
        elif k == 2:
            output = "W" 
        elif k == 3:
            output = "T"
        elif k == 4:
            output = "F"
            
        for j in range(n):
            output += f"   {int(x[i,j,k].x)}"
        print(output)

Department 0
   R1  R2  R3  R4  R5
M   0   0   1   1   0
T   0   0   1   1   0
W   0   0   1   1   0
T   0   0   1   1   0
F   1   1   0   0   0
Department 1
   R1  R2  R3  R4  R5
M   0   0   0   0   0
T   0   0   0   0   0
W   0   1   0   0   0
T   0   0   0   0   0
F   0   0   0   0   0
Department 2
   R1  R2  R3  R4  R5
M   0   0   0   0   1
T   0   1   0   0   0
W   0   0   0   0   1
T   1   1   0   0   0
F   0   0   1   1   0
Department 3
   R1  R2  R3  R4  R5
M   0   0   0   0   0
T   0   0   0   0   1
W   1   0   0   0   0
T   0   0   0   0   0
F   0   0   0   0   0
Department 4
   R1  R2  R3  R4  R5
M   0   1   0   0   0
T   0   0   0   0   0
W   0   0   0   0   0
T   0   0   0   0   0
F   0   0   0   0   1
Department 5
   R1  R2  R3  R4  R5
M   1   0   0   0   0
T   1   0   0   0   0
W   0   0   0   0   0
T   0   0   0   0   1
F   0   0   0   0   0
